In [ ]:
# 필요한 라이브러리 설치
!pip install face_recognition
!pip install ultralytics

# !pip install pytube

# # yt-dlp 설치
# !pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=654921f14b7e25dd50ce6f665bdeacf95b8ff2c5d15f249280ce5f01739f7c70
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 15.4 MB/s eta 0:00:00


In [ ]:
# yt-dlp 설치
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.6/171.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 79.1 MB/s eta 0:00:00


In [ ]:
# 필요한 라이브러리 설치
!pip install pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
import face_recognition
import cv2
import numpy as np
from ultralytics import YOLO
import torch
import pytube
from pytube import YouTube
from google.colab import files

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# youtube 영상 다운 (필요시)
linkAdd = "https://www.youtube.com/shorts/dfzl1XCjkDw"

In [ ]:
!yt-dlp linkAdd

[youtube] Extracting URL: https://www.youtube.com/shorts/dfzl1XCjkDw
[youtube] dfzl1XCjkDw: Downloading webpage
[youtube] dfzl1XCjkDw: Downloading ios player API JSON
[youtube] dfzl1XCjkDw: Downloading mweb player API JSON
[youtube] dfzl1XCjkDw: Downloading player 76c7a082
[youtube] dfzl1XCjkDw: Downloading m3u8 information
[info] dfzl1XCjkDw: Downloading 1 format(s): 247+251
[download] Destination: Tom Cruise Keeps Sending Everyone Cakes [dfzl1XCjkDw].f247.webm
[download] 100% of    4.45MiB in 00:00:00 at 26.41MiB/s
[download] Destination: Tom Cruise Keeps Sending Everyone Cakes [dfzl1XCjkDw].f251.webm
[download] 100% of  782.23KiB in 00:00:00 at 20.42MiB/s
[Merger] Merging formats into "Tom Cruise Keeps Sending Everyone Cakes [dfzl1XCjkDw].webm"
Deleting original file Tom Cruise Keeps Sending Everyone Cakes [dfzl1XCjkDw].f251.webm (pass -k to keep)
Deleting original file Tom Cruise Keeps Sending Everyone Cakes [dfzl1XCjkDw].f247.webm (pass -k to keep)


In [ ]:
# 비디오 처리 함수
def Face_Tracking(video, result, target_encoding):
    # model = YOLO('ysolov8n.pt')
    cap = cv2.VideoCapture(video)

    if not cap.isOpened():
        print("Error: 비디오 파일을 열 수 없습니다.")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(result, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 3프레임당 1번만 처리 (성능 향상)
        frame_count += 1
        if frame_count % 3 != 0:
            continue


        # # YOLO로 사람 탐지
        # results = model(frame)

        # # 프레임에서 얼굴 위치 찾기
        # face_locations = face_recognition.face_locations(frame)
        # face_encodings = face_recognition.face_encodings(frame, face_locations)



        # 프레임 크기 줄이기 (처리 속도 향상)
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)

        # 얼굴 위치 찾기
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)

        # 각 얼굴에 대해 처리
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # 크기를 원본에 맞게 조정
            top *= 2
            right *= 2
            bottom *= 2
            left *= 2

            matches = face_recognition.compare_faces([target_encoding], face_encoding)

            if matches[0]:
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, 'Target Person', (left, top - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # # 각 얼굴에 대해 처리
        # for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        #     matches = face_recognition.compare_faces([target_encoding], face_encoding)

        #     if matches[0]:
        #         cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        #         cv2.putText(frame, 'Target Person', (left, top - 10),
        #                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        out.write(frame)

    cap.release()
    out.release()

    print("처리 완료!")


In [ ]:
 # 타겟 인물의 얼굴 인코딩
target_image = face_recognition.load_image_file("TC.jpg")
target_encoding = face_recognition.face_encodings(target_image)[0]

In [ ]:
video_path=[]
output_path=[]

In [ ]:
# 실행
video_path0 = 'video0.webm'  # 현재 디렉토리의 video.webm 파일 사용
output_path0 = 'result_video0.mp4'

process_video(video_path0, output_path0, target_encoding)


100%|██████████| 6.25M/6.25M [00:00<00:00, 62.5MB/s]


처리 완료!


In [ ]:
video_path1 = 'video1.webm'  # 현재 디렉토리의 video.webm 파일 사용
output_path1 = 'result_video1.mp4'

process_video(video_path1, output_path1, target_encoding)

처리 완료!


In [ ]:
video_path2 = 'video2.webm'  # 현재 디렉토리의 video.webm 파일 사용
output_path2 = 'result_video2.mp4'

process_video(video_path2, output_path2, target_encoding)

처리 완료!


In [ ]:

# 결과 영상 다운로드
files.download(output_path0)
files.download(output_path1)
files.download(output_path2)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
video_path3 = 'video3.webm'  # 현재 디렉토리의 video.webm 파일 사용
output_path3 = 'result_video3.mp4'

process_video(video_path3, output_path3, target_encoding)

처리 완료!


In [ ]:
files.download(output_path3)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>